In [ ]:
import os
import torch

from torchvision.io import read_image
from torchvision.ops.boxes import masks_to_boxes
from torchvision import tv_tensors
from torchvision.transforms.v2 import functional as F
from google.colab import drive
from PIL import Image
import matplotlib.pyplot as plt
import sys
import pandas as pd
from collections import defaultdict
import json
import random
from torch.utils.data import Dataset
import numpy as np
import torchvision.models.detection as detection
from torch.utils.data import DataLoader
import torch.utils
import torch
from PIL import Image as read_image
from torchvision.io import read_image
from torchvision.transforms import v2
from transforms import RandomPhotometricDistort, RandomZoomOut, RandomIoUCrop, RandomHorizontalFlip,RandomVerticalFlip ,ToTensor, Compose
from torchvision.transforms.functional import convert_image_dtype

In [ ]:
drive.mount('/content/drive')
path_base = '/content/drive/MyDrive/Academico/TD8 personal '
sys.path.append(path_base)

Mounted at /content/drive


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:

train_file = path_base + '/classificationDownload (1)/train.csv'
test_file = path_base + '/classificationDownload (1)/test.csv'
classification_file = path_base + '/classificationDownload (1)/classifications.csv'

train_images = pd.read_csv(train_file)['image_filename'].tolist()
test_images = pd.read_csv(test_file)['image_filename'].tolist()


classification_data = pd.read_csv(classification_file)
train_data = classification_data[classification_data['image_filename'].isin(train_images)]
test_data = classification_data[classification_data['image_filename'].isin(test_images)]


def get_transform(train=True):
    transforms = []
    if train:
        transforms.append(RandomPhotometricDistort())
        transforms.append(RandomZoomOut(fill=list((207., 202., 205.))))
        transforms.append(RandomHorizontalFlip(p=0.5))
        transforms.append(RandomVerticalFlip(p=0.5))

    transforms.append(v2.ToDtype(torch.float, scale=True))
    return Compose(transforms)


def group_detections(data):
    grouped_data = {}
    for _, row in data.iterrows():
        image_name = row['image_filename']
        if image_name not in grouped_data:
            grouped_data[image_name] = {
                'image_filename': image_name,
                'image_id': row['image_id'],
                'image_doi': row['image_doi'],
                'classifications': []
            }
        grouped_data[image_name]['classifications'].append({
            'nucleus_x': row['nucleus_x'],
            'nucleus_y': row['nucleus_y'],
            'bethesda_system': row['bethesda_system'],
            'cell_id': row['cell_id']
        })
    return list(grouped_data.values())


train_data_grouped = group_detections(train_data)
test_data_grouped = group_detections(test_data)


train_counts = defaultdict(int)
test_counts = defaultdict(int)


for item in train_data_grouped:
    for classification in item['classifications']:
        train_counts[classification['bethesda_system']] += 1


for item in test_data_grouped:
    for classification in item['classifications']:
        test_counts[classification['bethesda_system']] += 1


class PAPDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

        self.label_map = {
            "SCC": 1,
            "HSIL": 2,
            "LSIL": 3,
            "ASC-H": 4,
            "ASC-US": 5,
            "Negative for intraepithelial lesion": 6
        }

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        img = read_image(path_base + '/Imagenes/' + item['image_filename'])

        image_doi = item['image_doi']
        image_id = item['image_id']
        boxes = []
        labels = []
        for classification in item['classifications']:
            nucleus_x = classification['nucleus_x']
            nucleus_y = classification['nucleus_y']
            x_min = nucleus_x - 50
            y_min = nucleus_y - 50
            x_max = nucleus_x + 50
            y_max = nucleus_y + 50
            boxes.append([x_min, y_min, x_max, y_max])
            labels.append(self.label_map[classification['bethesda_system']])


        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        target = {
            "boxes": boxes,
            "labels": labels,
            "area": area,
            "image_id": image_id
        }

        num_objs = boxes.shape[0]
        target["iscrowd"] = torch.zeros((num_objs,), dtype=torch.int64)

        if self.transform:
            img, target = self.transform(img, target)

        return img, target



train_dataset = PAPDataset(train_data_grouped, transform=get_transform(train=True))
test_dataset = PAPDataset(test_data_grouped, transform=get_transform(train=False))


print("Training Counts:", dict(train_counts))
print("Test Counts:", dict(test_counts))
print(f"Training Dataset Size: {len(train_dataset)}")
print(f"Test Dataset Size: {len(test_dataset)}")


Training Counts: {'SCC': 145, 'Negative for intraepithelial lesion': 6043, 'LSIL': 1221, 'HSIL': 1588, 'ASC-H': 808, 'ASC-US': 550}
Test Counts: {'ASC-H': 117, 'Negative for intraepithelial lesion': 736, 'ASC-US': 56, 'LSIL': 139, 'SCC': 16, 'HSIL': 115}
Training Dataset Size: 360
Test Dataset Size: 40


In [ ]:
from ssd import SSDFeatureExtractorResNet_Upgraded, ssd512_resnet50_Upgraded
model = ssd512_resnet50_Upgraded(pretrained = False, load = False)
model = model.to(device)



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 210MB/s]


In [ ]:
from ssd import SSDClassificationHead, SSDRegressionHead
num_classes = 6+1  # Ejemplo con 6  clases + 1 clase background


in_channels = [8192, 3584, 3584, 3328, 256, 256]
num_anchors = [4, 6, 6, 6, 4, 4]
model.head.classification_head = SSDClassificationHead(in_channels, num_anchors, num_classes)


In [ ]:

def collate_fn(batch):
    return tuple(zip(*batch))



batch_size = 16

train_sampler = torch.utils.data.RandomSampler(train_dataset)
test_sampler = torch.utils.data.SequentialSampler(test_dataset)

train_batch_sampler = train_batch_sampler = torch.utils.data.BatchSampler(
            train_sampler, batch_size, drop_last=True)


data_loader = torch.utils.data.DataLoader(
        train_dataset, batch_sampler=train_batch_sampler,
        collate_fn=collate_fn)


data_loader_test = torch.utils.data.DataLoader(
        test_dataset, batch_size=1,
        sampler=test_sampler,
        collate_fn=collate_fn)

In [ ]:
import math
from utils import MetricLogger, warmup_lr_scheduler, SmoothedValue
def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq):
    model.train()
    metric_logger = MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', SmoothedValue(window_size=1, fmt='{value:.6f}'))
    header = 'Epoch: [{}]'.format(epoch)

    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1. / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor)

    for images, targets in metric_logger.log_every(data_loader, print_freq, header):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) if not isinstance(v, int) else v for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        print(loss_value)

        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))
            break

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()

        metric_logger.update(lr=optimizer.param_groups[0]["lr"])

    return metric_logger



'\nMetricLogger te ayuda a monitorear métricas a lo largo del entrenamiento.\nWarmup Learning Rate Scheduler te permite empezar el entrenamiento de manera más suave ajustando gradualmente la tasa de aprendizaje durante las primeras iteraciones para evitar inestabilidad.\n'

In [ ]:
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_utils.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_eval.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/transforms.py
!pip install -U pycocotools

--2024-11-21 19:11:06--  https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4063 (4.0K) [text/plain]
Saving to: ‘engine.py’

engine.py           100%[===================>]   3.97K  --.-KB/s    in 0s      

2024-11-21 19:11:06 (86.0 MB/s) - ‘engine.py’ saved [4063/4063]

--2024-11-21 19:11:07--  https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8388 (8.2K) [text/plain]
S

In [ ]:
from torchvision.datasets import VOCDetection
from torchvision.transforms import Compose, ToTensor, RandomHorizontalFlip, Normalize
from engine import evaluate
import time
import datetime
import copy


params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
        params, lr = 0.003, momentum=0.9, weight_decay = 0.0005)

lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[80,110], gamma=0.1)

model.to(device)
best_ap = 0.0

print("Start training")
start_time = time.time()
for epoch in range(50):
  train_one_epoch(model, optimizer, data_loader, device, epoch, 20)
  lr_scheduler.step()
  ap = evaluate(model, data_loader_test, device=device).coco_eval["bbox"].stats[0]
  if ap > best_ap:
    best_ap = ap
    best_model_weights = copy.deepcopy(model.state_dict())

total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))








Start training
25.596172332763672
Epoch: [0]  [ 0/22]  eta: 0:08:01  lr: 0.000146  time: 21.8916  data: 18.7841  max mem: 11515
26.215299606323242
21.111108779907227
20.271451950073242
13.603216171264648
12.910543441772461
15.510641098022461
8.972517013549805
10.769674301147461
10.213947296142578
10.197393417358398
8.573317527770996
9.030025482177734
7.994481563568115
8.759180068969727
7.684913158416748
7.925560474395752
8.80788803100586
7.184103965759277
6.791423797607422
6.2359232902526855
Epoch: [0]  [20/22]  eta: 0:00:31  lr: 0.003000  time: 15.5617  data: 15.2144  max mem: 12451
6.456744194030762
Epoch: [0]  [21/22]  eta: 0:00:16  lr: 0.003000  time: 15.9773  data: 15.6305  max mem: 12451
Epoch: [0] Total time: 0:05:57 (16.2360 s / it)
creating index...
index created!
Test:  [ 0/40]  eta: 0:00:13  model_time: 0.2318 (0.2318)  evaluator_time: 0.0293 (0.0293)  time: 0.3292  data: 0.0639  max mem: 12451
Test:  [39/40]  eta: 0:00:00  model_time: 0.0168 (0.0224)  evaluator_time: 0.0210

In [ ]:
print('El AP del mejor modelo siendo esre el que guardamos es:')
print(best_ap)
torch.save({
    'model_state_dict': best_model_weights,
}, path_base + "/pesos_modelos/SSD512_ResNet_upgraded.pth" )

El AP del mejor modelo siendo esre el que guardamos es:
0.26405257133497645


In [ ]:
start_time = time.time()


for epoch in range(50, 100):
    train_one_epoch(model, optimizer, data_loader, device, epoch, 20)
    lr_scheduler.step()
    ap = evaluate(model, data_loader_test, device=device).coco_eval["bbox"].stats[0]
    if ap > best_ap:
      best_ap = ap
      best_model_weights = copy.deepcopy(model.state_dict())

total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))

2.193117380142212
Epoch: [50]  [ 0/22]  eta: 0:00:51  lr: 0.003000  time: 2.3202  data: 1.9500  max mem: 13131
2.5432889461517334
2.136230707168579
2.195775032043457
2.6506285667419434
2.213958263397217
2.101365327835083
2.1155691146850586
2.0874085426330566
2.5151844024658203
3.095442295074463
2.2977681159973145
1.9539971351623535
2.5592896938323975
2.4765257835388184
2.2791523933410645
2.3722035884857178
2.0098164081573486
2.2902040481567383
2.4596192836761475
2.081230640411377
Epoch: [50]  [20/22]  eta: 0:00:04  lr: 0.003000  time: 2.1918  data: 1.8489  max mem: 13131
2.3469972610473633
Epoch: [50]  [21/22]  eta: 0:00:02  lr: 0.003000  time: 2.2134  data: 1.8720  max mem: 13131
Epoch: [50] Total time: 0:00:48 (2.2092 s / it)
creating index...
index created!
Test:  [ 0/40]  eta: 0:00:04  model_time: 0.0209 (0.0209)  evaluator_time: 0.0149 (0.0149)  time: 0.1016  data: 0.0622  max mem: 13131
Test:  [39/40]  eta: 0:00:00  model_time: 0.0166 (0.0169)  evaluator_time: 0.0218 (0.0287)  ti

In [ ]:
print('El AP del mejor modelo siendo esre el que guardamos es:')
print(best_ap)
torch.save({
    'model_state_dict': best_model_weights,
}, path_base + "/pesos_modelos/SSD512_ResNet_upgraded.pth"  )


El AP del mejor modelo siendo esre el que guardamos es:
0.3246071879258207


In [ ]:
start_time = time.time()


for epoch in range(100, 120):
    train_one_epoch(model, optimizer, data_loader, device, epoch, 20)
    lr_scheduler.step()
    ap = evaluate(model, data_loader_test, device=device).coco_eval["bbox"].stats[0]
    if ap > best_ap:
      best_ap = ap
      best_model_weights = copy.deepcopy(model.state_dict())

total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))

1.8599072694778442
Epoch: [100]  [ 0/22]  eta: 0:00:51  lr: 0.000300  time: 2.3587  data: 1.9900  max mem: 13235
1.886842131614685
1.6973741054534912
1.7767308950424194
1.5606828927993774
1.5519896745681763
2.133807420730591
1.9044766426086426
1.9218721389770508
2.0035767555236816
1.9587440490722656
2.0900354385375977
1.6587753295898438
1.967315912246704
2.011277437210083
1.8178160190582275
1.9046727418899536
2.1060080528259277
1.6536223888397217
2.001002550125122
1.6102064847946167
Epoch: [100]  [20/22]  eta: 0:00:04  lr: 0.000300  time: 2.1399  data: 1.7999  max mem: 13235
1.448669195175171
Epoch: [100]  [21/22]  eta: 0:00:02  lr: 0.000300  time: 2.1254  data: 1.7923  max mem: 13235
Epoch: [100] Total time: 0:00:47 (2.1372 s / it)
creating index...
index created!
Test:  [ 0/40]  eta: 0:00:03  model_time: 0.0183 (0.0183)  evaluator_time: 0.0151 (0.0151)  time: 0.0991  data: 0.0620  max mem: 13235
Test:  [39/40]  eta: 0:00:00  model_time: 0.0164 (0.0169)  evaluator_time: 0.0192 (0.0275

In [ ]:
print('El AP del mejor modelo siendo esre el que guardamos es:')
print(best_ap)
torch.save({
    'model_state_dict': best_model_weights,
}, path_base + "/pesos_modelos/SSD512_ResNet_upgraded.pth"  )

El AP del mejor modelo siendo esre el que guardamos es:
0.3147435279110336


In [ ]:
start_time = time.time()


for epoch in range(120, 150):
    train_one_epoch(model, optimizer, data_loader, device, epoch, 20)
    lr_scheduler.step()
    ap = evaluate(model, data_loader_test, device=device).coco_eval["bbox"].stats[0]
    if ap > best_ap:
      best_ap = ap
      best_model_weights = copy.deepcopy(model.state_dict())

total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))

1.538367509841919
Epoch: [120]  [ 0/22]  eta: 0:00:40  lr: 0.000030  time: 1.8442  data: 1.4955  max mem: 13235
1.8297030925750732
1.5174823999404907
1.755579948425293
1.7185873985290527
1.8529560565948486
2.1071536540985107
1.7399592399597168
1.424063801765442
1.8535362482070923
2.0706257820129395
2.2389276027679443
1.9434089660644531
1.909554362297058
1.6347395181655884
1.420497179031372
1.8888683319091797
1.7660503387451172
1.9084970951080322
1.885814905166626
1.9345648288726807
Epoch: [120]  [20/22]  eta: 0:00:04  lr: 0.000030  time: 2.1913  data: 1.8311  max mem: 13235
1.607212781906128
Epoch: [120]  [21/22]  eta: 0:00:02  lr: 0.000030  time: 2.2054  data: 1.8465  max mem: 13235
Epoch: [120] Total time: 0:00:47 (2.1773 s / it)
creating index...
index created!
Test:  [ 0/40]  eta: 0:00:03  model_time: 0.0171 (0.0171)  evaluator_time: 0.0127 (0.0127)  time: 0.0952  data: 0.0617  max mem: 13235
Test:  [39/40]  eta: 0:00:00  model_time: 0.0168 (0.0168)  evaluator_time: 0.0206 (0.0271)

In [ ]:
print('El AP del mejor modelo siendo esre el que guardamos es:')
print(best_ap)
torch.save({
    'model_state_dict': best_model_weights,
}, path_base + "/pesos_modelos/SSD512_ResNet_upgraded.pth"  )

El AP del mejor modelo siendo esre el que guardamos es:
0.3246071879258207
